In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from diffusers import DiffusionPipeline
import torch

device='mps'
pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", variant="fp16", use_safetensors=True, torch_dtype=torch.float16).to(device)

## Directly using the pipeline

In [ ]:
image = pipeline(prompt="A cat playing with a ball in the forest", negative_prompt="deformed, ugly",
                 num_inference_steps=10, width=512, height=512).images[0]
image

## Using Compel, no weighting

In [ ]:
from compel import CompelForSDXL
compel = CompelForSDXL(pipeline)

conditioning = compel(["A cat playing with a ball in the forest", "deformed, ugly"])
image = pipeline(prompt_embeds=conditioning.embeds[0:1], pooled_prompt_embeds=conditioning.pooled_embeds[0:1],
                 negative_prompt_embeds=conditioning.embeds[1:2], negative_pooled_prompt_embeds=conditioning.pooled_embeds[1:2],
                 num_inference_steps=10, width=512, height=512).images[0]
image

## Using Compel, with weighting

In [ ]:
from compel import CompelForSDXL
compel = CompelForSDXL(pipeline)

conditioning = compel(["A cat playing with a ball in the forest", "deformed, ugly"])
image = pipeline(prompt_embeds=conditioning.embeds[0:1], pooled_prompt_embeds=conditioning.pooled_embeds[0:1],
                 negative_prompt_embeds=conditioning.embeds[1:2], negative_pooled_prompt_embeds=conditioning.pooled_embeds[1:2],
                 num_inference_steps=10, width=512, height=512).images[0]
image

## Long prompts

In [ ]:

prompt = "a cat playing with a ball++ in the forest"
negative_prompt = "a long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long negative prompt"
conditioning = compel([prompt, negative_prompt])
print(conditioning.embeds.shape, conditioning.pooled_embeds.shape)

image = pipeline(prompt_embeds=conditioning.embeds[0:1], pooled_prompt_embeds=conditioning.pooled_embeds[0:1],
                 negative_prompt_embeds=conditioning.embeds[1:2], negative_pooled_prompt_embeds=conditioning.pooled_embeds[1:2],
                 num_inference_steps=24, width=768, height=768).images[0]
image

## Sequential cpu offload

In [ ]:
from compel import Compel, ReturnedEmbeddingsType

compel = Compel(tokenizer=[pipeline.tokenizer, pipeline.tokenizer_2] , 
                text_encoder=[pipeline.text_encoder, pipeline.text_encoder_2], 
                returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, 
                requires_pooled=[False, True],
                device="cuda")

pipeline.enable_sequential_cpu_offload()
prompt = "a cat playing with a ball++ in the forest"
negative_prompt = "deformed, ugly"
conditioning, pooled = compel([prompt, negative_prompt])
print(conditioning.shape, pooled.shape)

image = pipeline(prompt_embeds=conditioning[0:1], pooled_prompt_embeds=pooled[0:1], 
                 negative_prompt_embeds=conditioning[1:2], negative_pooled_prompt_embeds=pooled[1:2],
                 num_inference_steps=24, width=768, height=768).images[0]
image

## Different prompts for different encoders

In [ ]:
prompt1 = "a cat playing with a ball++ in the forest"
prompt2 = "forest ambience, high quality, detailed, intricate, artstation, 8k"

compel1 = Compel(
    tokenizer=pipeline.tokenizer,
    text_encoder=pipeline.text_encoder,
    returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
    requires_pooled=False,
)

compel2 = Compel(
    tokenizer=pipeline.tokenizer_2,
    text_encoder=pipeline.text_encoder_2,
    returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
    requires_pooled=True,
)

conditioning1 = compel1(prompt1)
conditioning2, pooled = compel2(prompt2)
conditioning = torch.cat((conditioning1, conditioning2), dim=-1)

image = pipeline(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, num_inference_steps=30).images[0]